In [1]:
import pandas as pd
import numpy as np
from scipy import stats

In [2]:
app_launch = pd.read_csv('../input/app_launch_log.txt',header=None,sep='\t',names=['user_id','launch_day'])
user_activity = pd.read_csv('../input/user_activity_log.txt',header=None,sep='\t',names=['user_id','activity_day','page','video_id','author_id','action_type'])
user_register = pd.read_csv('../input/user_register_log.txt',header=None,sep='\t',names=['user_id','register_day','register_type','device_type'])
video_create = pd.read_csv('../input/video_create_log.txt',header=None,sep='\t',names=['user_id','create_day'])
app_launch.sort_values(by=['user_id','launch_day'],inplace=True)
user_activity.sort_values(by=['user_id','activity_day'],inplace=True)

In [3]:
data1_user=user_register[(user_register['register_day'] >= 1) & (user_register['register_day'] <= 16)]
data2_user=user_register[(user_register['register_day'] >= 1) & (user_register['register_day'] <= 23)]

In [4]:
print(data1_user.shape)
print(len(np.unique(data1_user['user_id'])))
print(data2_user.shape)
print(len(np.unique(data2_user['user_id'])))

(23848, 4)
23848
(37446, 4)
37446


In [3]:
temp_create = video_create[(video_create['create_day'] >= 1) & (video_create['create_day'] <= 16)]
temp_launch = app_launch[(app_launch['launch_day'] >= 1) & (app_launch['launch_day'] <= 16)]
temp_activity = user_activity[(user_activity['activity_day'] >= 1) & (user_activity['activity_day'] <= 16)]
data2_temp_create = video_create[(video_create['create_day'] >= 8) & (video_create['create_day'] <= 23)]
data2_temp_launch = app_launch[(app_launch['launch_day'] >= 8) & (app_launch['launch_day'] <= 23)]
data2_temp_activity = user_activity[(user_activity['activity_day'] >= 8) & (user_activity['activity_day'] <= 23)]

In [6]:
print(len(np.unique(pd.concat([temp_create['user_id'],temp_launch['user_id'],temp_activity['user_id']]))))
print(len(np.unique(pd.concat([data2_temp_create['user_id'],data2_temp_launch['user_id'],data2_temp_activity['user_id']]))))

23848
33992


In [10]:
# 极差
def ptp(column):
    return max(column) - min(column)
# 方差除以评率
def var_divide_count(column):
    return np.var(column) / len(column)

def last_one(column):
    if len(column) < 1:
        return column
    return column.iloc[-1]
def last_one_minus_reg():
    return

def q25(column):
    return column.quantile(.25)


def q75(column):
    return column.quantile(.75)


def kurt(column):
    return column.kurt()

In [12]:
# APP 启动次数，平均次数，方差，启动天数，最大值，最小值，连续几天启动总次数，平均次数，某一天启动次数
# agg要使用的函数
func = ['min', 'max', 'mean', 'median', 'skew', 'std', 'mad', 'count', ptp, np.var, var_divide_count, last_one, q25, q75, kurt]
app = temp_launch.groupby('user_id')['launch_day'].agg(func).reset_index()
app = app.rename(columns={'min': 'app_min', 'max': 'app_max', 'mean': 'app_mean', 'median': 'app_median',
             'skew': 'app_skew', 'std': 'app_std', 'mad': 'app_mad', 'count': 'app_count',
             'ptp': 'app_ptp', 'var': 'app_var', 'var_divide_count': 'app_var_divide_count',
             'last_one': 'app_last_one', 'q25': 'app_q25', 'q75': 'app_q75', 'kurt': 'app_kurt'})

In [13]:
app.head()

,user_id,app_min,app_max,app_mean,app_median,app_skew,app_std,app_mad,app_count,app_ptp,app_var,app_var_divide_count,app_last_one,app_q25,app_q75,app_kurt
0,16,13,15,14.00,14.0,0.000000,1.000000,0.666667,3,2,1.000000,0.222222,15,13.50,14.50,NaN
1,98,16,16,16.00,16.0,NaN,NaN,0.000000,1,0,NaN,0.000000,16,16.00,16.00,NaN
2,105,12,16,14.25,14.5,-0.752837,1.707825,1.250000,4,4,2.916667,0.546875,16,13.50,15.25,0.342857
3,218,11,16,13.50,13.5,0.000000,1.870829,1.500000,6,5,3.500000,0.486111,11,12.25,14.75,-1.200000
4,225,7,7,7.00,7.0,NaN,NaN,0.000000,1,0,NaN,0.000000,7,7.00,7.00,NaN


In [26]:
launch_day_diff = pd.concat([app_launch['user_id'], app_launch.groupby(['user_id']).diff().rename({'launch_day': 'launch_day_diff'}, axis=1)], axis=1)

In [44]:
launch_total_count = app_launch[['user_id']].groupby(['user_id']).size().rename('launch_total_count').reset_index()
launch_rest_day = (end_day - app_launch.groupby(['user_id'])['launch_day'].max() + 1).rename('launch_rest_day').reset_index()

In [50]:
    launch_rest_day['launch_average']=launch_total_count['launch_total_count'].div(launch_rest_day['launch_rest_day'])

In [46]:
launch_average=launch_total_count['launch_total_count']/launch_rest_day['launch_rest_day']

In [47]:
launch_average.head()

0     1.125000
1     0.142857
2     0.066667
3    15.000000
4     4.000000
dtype: float64

In [48]:
launch_total_count.head()

,user_id,launch_total_count
0,16,9
1,30,1
2,98,1
3,105,15
4,176,4


In [51]:
launch_rest_day.head()

,user_id,launch_rest_day,launch_average
0,16,8,1.125000
1,30,7,0.142857
2,98,15,0.066667
3,105,1,15.000000
4,176,1,4.000000


In [29]:
app_launch.groupby(['user_id']).diff().rename({'launch_day': 'launch_day_diff'}, axis=1)

,launch_day_diff
0,NaN
1,NaN
2,5.0
3,2.0
4,1.0
5,6.0
6,6.0
7,2.0
8,2.0
9,NaN


In [27]:
launch_day_diff.head()

,user_id,launch_day_diff
0,383135,NaN
1,330986,NaN
2,330986,5.0
3,330986,2.0
4,330986,1.0


In [6]:
userPre = user_activity[user_activity['activity_day']>=24]
sub = userPre[['user_id']].drop_duplicates()
sub.to_csv("../submission/baseline0611.csv",encoding='utf-8',index=None,header=None)

In [31]:

df1 = pd.DataFrame({'A':[1,1,2,2],'B':[2,2,6,8],'C':[8,7,6,5]})
df2 = pd.DataFrame({'A':[3,1,6,9,56],'B':[4,2,9,10,76],'D':[1,2,3,4,25]})
#df = df1.merge(df2,how='outer',on=['A','B'],left_index=True)
#t = df1.groupby('A').agg({'C':'min'})

#t = df1.groupby(['A'])['C'].min().rename('T').reset_index()


In [36]:
t = df1.groupby('A').agg({'C':'min'})

In [37]:
t

,C
A,
1,7
2,5


In [38]:
df1

,A,B,C
0,1,2,8
1,1,2,7
2,2,6,6
3,2,8,5


In [41]:
t = df1.groupby('A').agg({'A':'mean'})

In [15]:
end_day = np.max(temp_launch['launch_day'])
start_day = np.min(temp_launch['launch_day'])

In [6]:
temp_launch.head()

,user_id,launch_day
0,383135,1
1,330986,4
2,330986,9
3,330986,11
4,330986,12


In [137]:
test=pd.DataFrame()
test['launch_day']=[1,2,3,4,5,12,13,15]

In [138]:
test

,launch_day
0,1
1,2
2,3
3,4
4,5
5,12
6,13
7,15


In [142]:
end_day=15
start_day=1

In [145]:
sorted(test['launch_day'].tolist())

[1, 2, 3, 4, 5, 12, 13, 15]

In [147]:
seq[0]

16

In [143]:
seq = sorted(line['launch_day'].tolist())
length = end_day - seq[0] + 1
seq = [x - start_day + 1 for x in seq]

In [150]:
launch_day_diff = pd.concat([temp_launch['user_id'], temp_launch.groupby(['user_id']).diff().rename({'launch_day': 'launch_day_diff'}, axis=1)], axis=1)

In [151]:
launch_day_diff.head()

,user_id,launch_day_diff
44140,16,NaN
44141,16,1.0
212269,16,1.0
223219,98,NaN
125555,105,NaN


In [152]:
launch_day_diff.groupby(['user_id'])['launch_day_diff'].max().rename('launch_day_diff_max').reset_index()

,user_id,launch_day_diff_max
0,16,1.0
1,98,NaN
2,105,2.0
3,218,1.0
4,225,NaN
5,336,5.0
6,422,NaN
7,607,NaN
8,711,NaN
9,746,NaN


In [155]:
%%time
app_launch.groupby('user_id').agg([min,max])

Wall time: 48.9 ms


launch_day    
               min max
user_id               
16              13  23
30              24  24
98              16  16
105             12  30
176             27  30
211             25  25
218             11  30
225              7  29
250             17  29
325             22  30
336              1  30
341             25  27
420             26  28
422              5   5
481             23  29
493             23  30
555             21  30
581             26  30
589             27  28
607              1   1
639             18  18
662             25  27
674             29  30
711              6  29
746             14  14
820             23  26
858             21  21
901             14  22
927             28  28
973              4  30
...            ...  ..
1366683         21  30
1366711          2   2
1366722         24  24
1366737         29  30
1366798         22  30
1366803         18  24
1366822         28  29
1366834          6  29
1366874         24  28
1366899          3  29
1366920         12  12
1366989         16  16
1367006         17  17
1367044          1  30
1367067         10  27
1367106         29  30
1367112         21  22
1367164         16  16
1367252         16  28
1367259         26  28
1367268         30  30
1367348         10  10
1367360         20  20
1367419         26  26
1367421          6  30
1367449         23  29
1367473          4   5
1367496         30  30
1367532         15  30
1367534         24  29

[51709 rows x 2 columns]

In [158]:
def test2(seq, user_id):
    df = pd.DataFrame()
    df['user_id'] = [user_id]
    df['max']=np.max(seq)
    df['min']=np.min(seq)
    return df

def test(line):
    user_id = line['user_id'].tolist()[0]
    test2(seq, user_id)

In [162]:
%%time
test = app_launch.groupby(['user_id'])['launch_day'].agg({'launch_day_duration': lambda x: (end_day - min(x) + 1)}).reset_index()

D:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  """Entry point for launching an IPython kernel.


Wall time: 668 ms


In [ ]:
%%time
test = app_launch.groupby(['user_id'])['launch_day'].agg({'launch_day_duration': lambda x: (end_day - min(x) + 1)}).reset_index()

In [1]:
from itertools import groupby
from operator import itemgetter

# 连续多少天启动
def get_continuous(data):
    ranges = []
    for k, g in groupby(enumerate(data), lambda x: x[0] - x[1]):
        group = (map(itemgetter(1), g))
        group = list(map(int, group))
        ranges.append(group[-1] - group[0] + 1)
    return max(ranges)

In [8]:
app_launch.groupby(['user_id'])['launch_day'].agg([get_continuous]).reset_index().rename()

,user_id,get_continuous
0,16,3
1,30,1
2,98,1
3,105,3
4,176,4
5,211,1
6,218,3
7,225,3
8,250,3
9,325,1


In [13]:
app_launch.groupby(['user_id']).size().rename('launch_total_count').reset_index()

,user_id,launch_total_count
0,16,9
1,30,1
2,98,1
3,105,15
4,176,4
5,211,1
6,218,19
7,225,14
8,250,12
9,325,4


In [14]:
app_launch.groupby(['user_id'])['launch_day'].size().rename('launch_total_count').reset_index()

,user_id,launch_total_count
0,16,9
1,30,1
2,98,1
3,105,15
4,176,4
5,211,1
6,218,19
7,225,14
8,250,12
9,325,4


In [ ]:
launch_day_diff = pd.concat([app_launch['user_id'], ], axis=1)

In [24]:
%%time
app_launch.groupby(['user_id']).diff().rename({'launch_day': 'launch_day_diff'}, axis=1)

Wall time: 13.8 s


,launch_day_diff
44140,NaN
44141,1.0
212269,1.0
212270,3.0
212271,1.0
212272,1.0
127816,1.0
212273,1.0
212274,1.0
75755,NaN


In [23]:
%%time
pd.concat([app_launch['user_id'], app_launch.groupby(['user_id']).diff().rename({'launch_day': 'launch_day_diff'}, axis=1)], axis=1)

Wall time: 14.2 s


,user_id,launch_day_diff
44140,16,NaN
44141,16,1.0
212269,16,1.0
212270,16,3.0
212271,16,1.0
212272,16,1.0
127816,16,1.0
212273,16,1.0
212274,16,1.0
75755,30,NaN


In [21]:
app_launch[app_launch['user_id']==330986]

,user_id,launch_day
83666,330986,1
83667,330986,2
167756,330986,3
1,330986,4
167757,330986,5
167758,330986,6
167759,330986,7
83668,330986,8
2,330986,9
167760,330986,10


In [25]:
app_launch[app_launch['user_id']==330986].diff()

,user_id,launch_day
83666,NaN,NaN
83667,0.0,1.0
167756,0.0,1.0
1,0.0,1.0
167757,0.0,1.0
167758,0.0,1.0
167759,0.0,1.0
83668,0.0,1.0
2,0.0,1.0
167760,0.0,1.0


In [26]:
video_create.sort_values(by=['user_id','create_day'])

,user_id,create_day
7951,98,16
9815,555,22
9814,555,24
21855,973,13
21849,973,14
21851,973,14
21852,973,14
21844,973,20
21843,973,25
21845,973,27


In [28]:
video_create.groupby(['user_id'])['create_day'].agg({'launch_day_duration': lambda x: len(np.unique(x))})

D:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  """Entry point for launching an IPython kernel.


,launch_day_duration
user_id,
98,1
555,2
973,6
1014,6
1180,1
1296,5
1400,1
1938,1
2040,3


In [30]:
video_create.groupby(['user_id'])['create_day'].agg('count')

user_id
98          1
555         2
973        15
1014        6
1180        1
1296        9
1400        4
1938        1
2040        8
2068        1
2276        5
2306        1
2439       17
2914       14
2941       10
3473        1
3650        3
3883        1
3897        6
3952        4
4019        3
4370        2
4527       51
4790        5
4800        2
4831        1
5527        2
5789        5
5861       17
5989        4
           ..
1362387     1
1362746    16
1362781     1
1362826    11
1362866     1
1363730     5
1363930     1
1364055     1
1364245     1
1364263     3
1364358     6
1364467     1
1364756     1
1364850     4
1364947     1
1365465     2
1365674     1
1366109     1
1366113     3
1366200     1
1366299     3
1366305     2
1366339     1
1366419     5
1366491     1
1366798     6
1366834     7
1367067     1
1367112     7
1367473    12
Name: create_day, Length: 7606, dtype: int64

In [4]:
user_activity.sort_values(by=['user_id','activity_day'],inplace=True)

,user_id,activity_day,page,video_id,author_id,action_type
496070,16,13,3,2911602,245430,0
522440,16,13,3,1066348,245430,0
522441,16,13,3,1066348,245430,0
815812,16,13,3,1277921,280089,0
1289214,16,13,3,2565669,280089,0


In [5]:
user_activity.head(15)

,user_id,activity_day,page,video_id,author_id,action_type
496070,16,13,3,2911602,245430,0
522440,16,13,3,1066348,245430,0
522441,16,13,3,1066348,245430,0
815812,16,13,3,1277921,280089,0
1289214,16,13,3,2565669,280089,0
2131571,16,13,3,2653595,280089,0
4277700,16,13,3,3355338,280089,0
4550265,16,13,3,3860100,280089,0
6024364,16,13,3,1747202,280089,0
7160265,16,13,3,1957749,103001,0


In [4]:
# cre = pd.read_csv('d://cretest.csv')
# act = pd.read_csv("d://acttest.csv")
def get_cre_feature(row):
    user = list(row["user_id"])[0]
    feature = pd.Series()
    feature["user_id"]=user
    if user in watch_sum_count:
        feature['wacth_sum_count'] = watch_sum_count[user]
        feature['watch_num_people'] = watch_num_people[user]

    else:
        feature['wacth_sum_count'] = 0
        feature['watch_num_people'] = 0

        
    return feature
creater=np.unique(temp_create['user_id'])
watch_sum_count={}
watch_num_people={}

def get_watch_user_feature(row):
    feature=pd.Series()
    feature['user_id']=list(row["user_id"])[0]
    feature['user_count']=len(row["user_id"])
    feature['user_day']=len(np.unique(row["activity_day"]))
    
    return feature

def get_watch_video_feature(row):
    feature=pd.Series()
    feature['user_id']=list(row["user_id"])[0]
    feature['video_count']=len(row["user_id"])
    feature['video_day']=len(np.unique(row["activity_day"]))
    
    return feature

# act.groupby("author_id",sort=True).apply(get_author_feature)
feature=temp_create.groupby("user_id",sort=True).apply(get_cre_feature)
print(feature)

         user_id  wacth_sum_count  watch_num_people
user_id                                            
98            98                0                 0
973          973                0                 0
1014        1014                0                 0
1296        1296                0                 0
1938        1938                0                 0
2040        2040                0                 0
2068        2068                0                 0
2276        2276                0                 0
2439        2439                0                 0
2914        2914                0                 0
3883        3883                0                 0
3897        3897                0                 0
4019        4019                0                 0
4370        4370                0                 0
4527        4527                0                 0
4790        4790                0                 0
5861        5861                0                 0
5989        

In [5]:
# cre = pd.read_csv('d://cretest.csv')
# act = pd.read_csv("d://acttest.csv")
def get_cre_feature(row):
    user = list(row["user_id"])[0]
    feature = pd.Series()
    feature["user_id"]=user
    if user in watch_sum_count:
        feature['wacth_sum_count'] = watch_sum_count[user]
        feature['watch_num_people'] = watch_num_people[user]

    else:
        feature['wacth_sum_count'] = 0
        feature['watch_num_people'] = 0

        
    return feature
creater=np.unique(temp_create['user_id'])
watch_sum_count={}
watch_num_people={}

def get_watch_user_feature(row):
    feature=pd.Series()
    feature['user_id']=list(row["user_id"])[0]
    feature['user_count']=len(row["user_id"])
    feature['user_day']=len(np.unique(row["activity_day"]))
    
    return feature

def get_watch_video_feature(row):
    feature=pd.Series()
    feature['user_id']=list(row["user_id"])[0]
    feature['video_count']=len(row["user_id"])
    feature['video_day']=len(np.unique(row["activity_day"]))
    
    return feature

def get_author_feature(row):
    global creater
    # print(author[0])
    author=list(row["author_id"])[0]
    if author in creater:
        watch_sum_count[author] = len(row["user_id"])
        watch_num_people[author] = len(np.unique(row["user_id"]))

        # print(row)
        user_feature = row.groupby("user_id", sort=True).apply(get_watch_user_feature)
        video_feature=row.groupby("user_id", sort=True).apply(get_watch_video_feature)
                # print(user_feature)

feature = temp_activity.groupby("author_id",sort=True).apply(get_author_feature)
# feature=temp_create.groupby("user_id",sort=True).apply(get_cre_feature)
print(feature)

Empty DataFrame
Columns: []
Index: []


In [67]:
weekend_count=temp_activity.groupby(['user_id'])['activity_day'].agg(
        {'activity_weekend_day_count': lambda x: len([day for day in x if day % 7 == 0 or (day + 1) % 7 == 0])})#在周末的总共活动次数

D:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  


In [47]:
for i in range(5):
    weekend_count['all_weekend_count_page_%d'%i]=temp_activity[temp_activity['page']==i].groupby(['user_id'])['activity_day'].agg(
        {lambda x: len([day for day in x if day % 7 == 0 or (day + 1) % 7 == 0])})#在周末的总共活动次数

In [185]:
end_day = np.max(temp_activity['activity_day'])
start_day = np.min(temp_activity['activity_day'])

In [59]:
last_seven_activity_count['last_five_activity_count']=temp_activity[(temp_activity['activity_day'] >= end_day - 4) & (temp_activity['activity_day'] <= end_day)].groupby(['user_id'])[
        'activity_day'].size()
# last_five_activity_count.head()

In [60]:
last_seven_activity_count.head()

,user_id,last_seven_activity_count,last_five_activity_count
0,16,57,NaN
1,98,1,NaN
2,105,294,NaN
3,218,343,NaN
4,336,50,NaN


In [86]:
print(weekend_count.shape)
weekend_count.head()

(20478, 11)


,activity_weekend_day_count,page_0_all,page_0_percent,page_1_all,page_1_percent,page_2_all,page_2_percent,page_3_all,page_3_percent,page_4_all,page_4_percent
user_id,,,,,,,,,,,
16,48,36.0,0.631579,NaN,NaN,8.0,0.140351,13.0,0.228070,NaN,NaN
98,0,NaN,NaN,NaN,NaN,1.0,1.000000,NaN,NaN,NaN,NaN
105,119,272.0,0.925170,2.0,0.006803,20.0,0.068027,NaN,NaN,NaN,NaN
218,32,69.0,0.201166,89.0,0.259475,NaN,NaN,183.0,0.533528,2.0,0.005831
225,30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30.0,1.000000


In [150]:
# 统计 page 和 type | 比例
def other_count(df, others, feature_name, other_name):
    others = pd.DataFrame(others.groupby(['user_id', feature_name])[feature_name].count()).reset_index('user_id')
    others.rename(columns={feature_name: feature_name + '_count'}, inplace=True)
    others = others.reset_index(feature_name)
    others = others.pivot('user_id', feature_name, feature_name + '_count').reset_index('user_id')
    columns = others.columns.tolist()
    percent_lists = []
    for c in columns:
        if c is not 'user_id':
            others.rename(columns={c: feature_name + '_' + str(c) + '_' + other_name}, inplace=True)
            percent_lists.append(feature_name + '_' + str(c) + '_' + other_name)
    df = pd.merge(df, others, how='left', on='user_id')
    # extract percentage feature
    for c in percent_lists:
        df[c + '_percent'] = df[c] / df[other_name + '_action_count']
    return df

In [85]:
%%time
# activity_total_count = temp_activity.groupby(['user_id']).size().rename('activity_total_count').reset_index()  # 总活动次数
for i in range(5):
    weekend_count['page_%d_all' % i] = temp_activity[temp_activity['page'] == i].groupby(['user_id'])['page'].size()
    weekend_count['page_%d_percent' % i] = weekend_count['page_%d_all' % i] / activity_total_count['activity_total_count']

#     activity_total_count['page_%d_percent' % i] = activity_total_count['page_%d_all' % i] / activity_total_count['activity_total_count']

Wall time: 538 ms


In [104]:
df.head()

page,user_id,page_0_last_seven,page_1_last_seven,page_2_last_seven,page_3_last_seven,page_4_last_seven
0,16,36.0,NaN,8.0,13.0,NaN
1,98,NaN,NaN,1.0,NaN,NaN
2,105,272.0,2.0,20.0,NaN,NaN
3,218,69.0,89.0,NaN,183.0,2.0
4,336,32.0,NaN,NaN,18.0,NaN


In [159]:
activity_total_count = temp_activity.groupby(['user_id'])['activity_day'].agg({'all_action_count':'count'}) # 总活动次数
# activity_total_count['last_seven_activity_count'] = activity[(activity['activity_day'] >= end_day - 6) & (activity['activity_day'] <= end_day)].groupby(['user_id'])['activity_day'].size()#
# action 总次数, 最后 7 5 3 1
# activity_total_count = activity.groupby(['user_id'])['activity_day'].agg({'all_action_count': 'count'})  # 总活动次数
activity_total_count['last_seven_action_count'] = activity[(activity['activity_day'] >= end_day - 6) & (activity['activity_day'] <= end_day)].groupby(['user_id'])['activity_day'].size()  #
activity_total_count['last_five_action_count'] = activity[(activity['activity_day'] >= end_day - 4) & (activity['activity_day'] <= end_day)].groupby(['user_id'])['activity_day'].size()
activity_total_count['last_three_action_count'] = activity[(activity['activity_day'] >= end_day - 2) & (activity['activity_day'] <= end_day)].groupby(['user_id'])['activity_day'].size()
activity_total_count['last_one_action_count'] = activity[(activity['activity_day'] == end_day)].groupby(['user_id'])['activity_day'].size()

D:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  """Entry point for launching an IPython kernel.


In [98]:
activity_total_count['last_seven_activity_count'] = temp_activity[(temp_activity['activity_day'] >= end_day - 6) & (temp_activity['activity_day'] <= end_day)].groupby(['user_id'])['activity_day'].size()#.rename('last_seven_activity_count').reset_index()


In [247]:
activity=temp_activity

In [112]:
%%time
for i in range(5):
    activity_total_count['page_%d_all' % i] = activity[activity['page'] == i].groupby(['user_id'])['page'].size()
    activity_total_count['page_%d_percent' % i] = activity_total_count['page_%d_all' % i] / activity_total_count['all_action_count']

    activity_total_count['page_%d_last_seven_count' % i] = activity[(activity['page'] == i)&(activity['activity_day']>=end_day-6)&(activity['activity_day']<=end_day)].groupby(['user_id'])['page'].size()
    activity_total_count['page_%d_last_seven_percent' % i] = activity_total_count['page_%d_last_seven_count' % i] / activity_total_count['last_seven_activity_count']

Wall time: 1.17 s


In [90]:
activity_total_count['page_0_all'] = temp_activity[temp_activity['page'] == 0].groupby(['user_id'])['page'].size()

In [89]:
temp=temp_activity[temp_activity['page'] == 0].groupby(['user_id'])['page'].size()
temp.head()

user_id
16      36
105    272
218     69
336     60
711      5
Name: page, dtype: int64

In [113]:
print(activity_total_count.shape)
activity_total_count.head()

(20478, 22)


,all_action_count,last_seven_activity_count,page_0_all,page_0_percent,page_0_last_seven_count,page_0_last_seven_percent,page_1_all,page_1_percent,page_1_last_seven_count,page_1_last_seven_percent,...,page_2_last_seven_count,page_2_last_seven_percent,page_3_all,page_3_percent,page_3_last_seven_count,page_3_last_seven_percent,page_4_all,page_4_percent,page_4_last_seven_count,page_4_last_seven_percent
user_id,,,,,,,,,,,,,,,,,,,,,
16,57,57.0,36.0,0.631579,36.0,0.631579,NaN,NaN,NaN,NaN,...,8.0,0.140351,13.0,0.228070,13.0,0.228070,NaN,NaN,NaN,NaN
98,1,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
105,294,294.0,272.0,0.925170,272.0,0.925170,2.0,0.006803,2.0,0.006803,...,20.0,0.068027,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
218,343,343.0,69.0,0.201166,69.0,0.201166,89.0,0.259475,89.0,0.259475,...,NaN,NaN,183.0,0.533528,183.0,0.533528,2.0,0.005831,2.0,0.005831
225,30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,30.0,1.000000,NaN,NaN


In [ ]:
activity_total_count['page_0_all']

In [88]:
temp_activity[(temp_activity['user_id']==16) & (temp_activity['page']==0)].shape

(36, 6)

In [115]:
# 统计 video_id , author_id
def video_count(df, action, feature_name, other_name):
    def getVideo(l):
        return len(np.unique(l))

    temp = action.groupby(['user_id', feature_name])[feature_name].count().reset_index('user_id')
    temp.rename(columns={feature_name: feature_name + '_count' + '_' + other_name}, inplace=True)
    temp = temp.reset_index(feature_name)
    temp_num = temp.copy()
    temp = temp.groupby(['user_id']).agg('max').reset_index('user_id')[
        ['user_id', feature_name + '_count' + '_' + other_name]]
    temp_num = temp_num.groupby('user_id').agg(getVideo).reset_index('user_id')[
        ['user_id', feature_name]]
    temp_num.rename(columns={feature_name: feature_name + '_' + other_name}, inplace=True)
    # 最多的那个video收看了多少次
    df = pd.merge(df, temp, how='left', on='user_id')
    # 一共收看了多少个video
    df = pd.merge(df, temp_num, how='left', on='user_id')
    df[feature_name + '_count' + '_' + other_name + '_percent'] = df[feature_name + '_count' + '_' + other_name] / df[
        other_name + '_action_count']
    df[feature_name + '_' + other_name + '_percent'] = df[feature_name + '_' + other_name] / df[
        other_name + '_action_count']
    return df

In [136]:
%%time
# video 看了几个，看的最多的那个video有多少次action ，author 同 video
all_type = ['author_id', 'video_id']
for i in range(2):
    activity_total_count = video_count(activity_total_count, activity, all_type[i], 'all')
    activity_total_count = video_count(activity_total_count, activity[(activity['activity_day'] >= end_day - 6) & (activity['activity_day'] <= end_day)], all_type[i], 'last_seven')
    activity_total_count = video_count(activity_total_count, activity[(activity['activity_day'] >= end_day - 4) & (activity['activity_day'] <= end_day)], all_type[i], 'last_five')
    activity_total_count = video_count(activity_total_count, activity[(activity['activity_day'] >= end_day - 2) & (activity['activity_day'] <= end_day)], all_type[i], 'last_three')
    activity_total_count = video_count(activity_total_count, activity[activity['activity_day'] == end_day], all_type[i], 'last_one')

Wall time: 19.8 s


In [137]:
activity_total_count.head()

,user_id,all_action_count,last_seven_action_count,last_five_action_count,last_three_action_count,last_one_action_count,author_id_count_all,author_id_all,author_id_count_all_percent,author_id_all_percent,...,video_id_count_last_five_percent,video_id_last_five_percent,video_id_count_last_three,video_id_last_three,video_id_count_last_three_percent,video_id_last_three_percent,video_id_count_last_one,video_id_last_one,video_id_count_last_one_percent,video_id_last_one_percent
0,16,57,57.0,57.0,30.0,NaN,8,36,0.140351,0.631579,...,0.052632,0.824561,2.0,26.0,0.066667,0.866667,NaN,NaN,NaN,NaN
1,98,1,1.0,1.0,1.0,1.0,1,1,1.000000,1.000000,...,1.000000,1.000000,1.0,1.0,1.000000,1.000000,1.0,1.0,1.000000,1.000000
2,105,294,294.0,294.0,132.0,NaN,11,171,0.037415,0.581633,...,0.027211,0.612245,5.0,96.0,0.037879,0.727273,NaN,NaN,NaN,NaN
3,218,343,343.0,340.0,154.0,14.0,43,76,0.125364,0.221574,...,0.014706,0.811765,5.0,128.0,0.032468,0.831169,3.0,11.0,0.214286,0.785714
4,225,30,NaN,NaN,NaN,NaN,7,19,0.233333,0.633333,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [139]:
activity_total_count.head()

,all_action_count,last_seven_action_count,last_five_action_count,last_three_action_count,last_one_action_count
user_id,,,,,
16,57,57.0,57.0,30.0,NaN
98,1,1.0,1.0,1.0,1.0
105,294,294.0,294.0,132.0,NaN
218,343,343.0,340.0,154.0,14.0
225,30,NaN,NaN,NaN,NaN


In [152]:
%%time
# 总的Page 0 1 2 3 4 个数,最后7,5,3,1 | 总的action_type 0 1 2 3 4 5 个数，最后7,5,3,1
all_type = ['page', 'action_type']
for i in range(2):
    activity_total_count = other_count(activity_total_count, activity, all_type[i], 'all')
    activity_total_count = other_count(activity_total_count, activity[(activity['activity_day'] >= end_day - 6) & (activity['activity_day'] <= end_day)], all_type[i], 'last_seven')
    activity_total_count = other_count(activity_total_count, activity[(activity['activity_day'] >= end_day - 4) & (activity['activity_day'] <= end_day)], all_type[i], 'last_five')
    activity_total_count = other_count(activity_total_count, activity[(activity['activity_day'] >= end_day - 2) & (activity['activity_day'] <= end_day)], all_type[i], 'last_three')
    activity_total_count = other_count(activity_total_count, activity[activity['activity_day'] == end_day], all_type[i], 'last_one')

Wall time: 3.74 s


In [153]:
activity_total_count.head()

,user_id,all_action_count,last_seven_action_count,last_five_action_count,last_three_action_count,last_one_action_count,page_0_all,page_1_all,page_2_all,page_3_all,...,action_type_2_last_one,action_type_3_last_one,action_type_4_last_one,action_type_5_last_one,action_type_0_last_one_percent,action_type_1_last_one_percent,action_type_2_last_one_percent,action_type_3_last_one_percent,action_type_4_last_one_percent,action_type_5_last_one_percent
0,16,57,57.0,57.0,30.0,NaN,36.0,NaN,8.0,13.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,98,1,1.0,1.0,1.0,1.0,NaN,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
2,105,294,294.0,294.0,132.0,NaN,272.0,2.0,20.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,218,343,343.0,340.0,154.0,14.0,69.0,89.0,NaN,183.0,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
4,225,30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [157]:
activity_total_count.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20478 entries, 0 to 20477
Columns: 116 entries, user_id to action_type_5_last_one_percent
dtypes: float64(114), int64(2)
memory usage: 18.3 MB


In [148]:
%%time
# 总的Page 0 1 2 3 4 个数,最后7,5,3,1 | 总的action_type 0 1 2 3 4 5 个数，最后7,5,3,1
for i in range(5):
    activity_total_count['page_%d_all' % i] = activity[activity['page'] == i].groupby(['user_id'])['page'].size()
    activity_total_count['page_%d_percent' % i] = activity_total_count['page_%d_all' % i] / activity_total_count['all_action_count']
    activity_total_count['page_%d_last_seven_count' % i] = activity[(activity['page'] == i) & (activity['activity_day'] >= end_day - 6) & (activity['activity_day'] <= end_day)].groupby(['user_id'])['page'].size()
    activity_total_count['page_%d_last_seven_percent' % i] = activity_total_count['page_%d_last_seven_count' % i] / activity_total_count['last_seven_action_count']
    activity_total_count['page_%d_last_five_count' % i] = activity[(activity['page'] == i) & (activity['activity_day'] >= end_day - 4) & (activity['activity_day'] <= end_day)].groupby(['user_id'])['page'].size()
    activity_total_count['page_%d_last_five_percent' % i] = activity_total_count['page_%d_last_five_count' % i] / activity_total_count['last_five_action_count']
    activity_total_count['page_%d_last_three_count' % i] = activity[(activity['page'] == i) & (activity['activity_day'] >= end_day - 2) & (activity['activity_day'] <= end_day)].groupby(['user_id'])['page'].size()
    activity_total_count['page_%d_last_three_percent' % i] = activity_total_count['page_%d_last_three_count' % i] / activity_total_count['last_three_action_count']
    activity_total_count['page_%d_last_one_count' % i] = activity[(activity['page'] == i) & (activity['activity_day'] == end_day)].groupby(['user_id'])['page'].size()
    activity_total_count['page_%d_last_one_percent' % i] = activity_total_count['page_%d_last_one_count' % i] / activity_total_count['last_one_action_count']
for i in range(6):
    activity_total_count['type_%d_all' % i] = activity[activity['action_type'] == i].groupby(['user_id'])['action_type'].size()
    activity_total_count['type_%d_percent' % i] = activity_total_count['type_%d_all' % i] / activity_total_count['all_action_count']
    activity_total_count['type_%d_last_seven_count' % i] = activity[(activity['action_type'] == i) & (activity['activity_day'] >= end_day - 6) & (activity['activity_day'] <= end_day)].groupby(['user_id'])['action_type'].size()
    activity_total_count['type_%d_last_seven_percent' % i] = activity_total_count['type_%d_last_seven_count' % i] / activity_total_count['last_seven_action_count']
    activity_total_count['type_%d_last_five_count' % i] = activity[(activity['action_type'] == i) & (activity['activity_day'] >= end_day - 4) & (activity['activity_day'] <= end_day)].groupby(['user_id'])['action_type'].size()
    activity_total_count['type_%d_last_five_percent' % i] = activity_total_count['type_%d_last_five_count' % i] / activity_total_count['last_five_action_count']
    activity_total_count['type_%d_last_three_count' % i] = activity[(activity['action_type'] == i) & (activity['activity_day'] >= end_day - 2) & (activity['activity_day'] <= end_day)].groupby(['user_id'])['action_type'].size()
    activity_total_count['type_%d_last_three_percent' % i] = activity_total_count['type_%d_last_three_count' % i] / activity_total_count['last_three_action_count']
    activity_total_count['type_%d_last_one_count' % i] = activity[(activity['action_type'] == i) & (activity['activity_day'] == end_day)].groupby(['user_id'])['action_type'].size()
    activity_total_count['type_%d_last_one_percent' % i] = activity_total_count['type_%d_last_one_count' % i] / activity_total_count['last_one_action_count']

Wall time: 4.74 s


In [149]:
activity_total_count.head()

,all_action_count,last_seven_action_count,last_five_action_count,last_three_action_count,last_one_action_count,page_0_all,page_0_percent,page_0_last_seven_count,page_0_last_seven_percent,page_0_last_five_count,...,type_5_all,type_5_percent,type_5_last_seven_count,type_5_last_seven_percent,type_5_last_five_count,type_5_last_five_percent,type_5_last_three_count,type_5_last_three_percent,type_5_last_one_count,type_5_last_one_percent
user_id,,,,,,,,,,,,,,,,,,,,,
16,57,57.0,57.0,30.0,NaN,36.0,0.631579,36.0,0.631579,36.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
98,1,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
105,294,294.0,294.0,132.0,NaN,272.0,0.925170,272.0,0.925170,272.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
218,343,343.0,340.0,154.0,14.0,69.0,0.201166,69.0,0.201166,69.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
225,30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [165]:
def continous_day(df, action, other_name):
    def longest_day(seq):
        count = 1
        longest = 1
        last_one = seq[0]
        for i in range(len(seq)):
            if i + 1 < len(seq):
                if seq[i] + 1 == seq[i + 1]:
                    count += 1
                    if count > longest:
                        longest = count
                        last_one = seq[i + 1]
                else:
                    count = 1
        return [longest, last_one]

    def most_action(df):
        seq_day = df.activity_day
        seq_day_count = df.day_count
        max_day_count = 0
        max_day = 0
        for x, y in zip(seq_day, seq_day_count):
            if y > max_day_count:
                max_day = x
                max_day_count = y
        return max_day

    page = action.groupby(['user_id', 'activity_day'])['activity_day'].count()
    page = page.reset_index(['user_id'])
    page.rename(columns={'activity_day': 'day_count'}, inplace=True)
    page = page.reset_index('activity_day')
    temp_df = pd.DataFrame({
        'user_id': np.unique(page.user_id).tolist()
    })
    longest = []
    longest_last = []
    most_action_date = []
    for user in temp_df.user_id:
        piece_df = page[page.user_id == user]
        longest.append(longest_day(piece_df.activity_day.tolist())[0])
        longest_last.append(longest_day(piece_df.activity_day.tolist())[1])
        most_action_date.append(most_action(piece_df))
    temp_df[other_name + '_longest_action_day'] = longest
    temp_df[other_name + '_longest_action_day_last'] = longest_last
    temp_df[other_name + '_max_action_day'] = most_action_date
    df = pd.merge(df, temp_df, how='left', on='user_id')
    page = page.groupby('user_id')['day_count'].agg('max')
    page = page.reset_index('user_id')
    page.rename(columns={'day_count': other_name + '_max_action_one_day'}, inplace=True)
    df = pd.merge(df, page, how='left', on='user_id')
    return df

In [166]:
%%time
activity_total_count = continous_day(activity_total_count, activity, 'all')

Wall time: 10.8 s


In [167]:
activity_total_count.head()

,user_id,all_action_count,last_seven_action_count,last_five_action_count,last_three_action_count,last_one_action_count,all_longest_action_day,all_longest_action_day_last,all_max_action_day,all_max_action_one_day
0,16,57,57.0,57.0,30.0,NaN,3,15,13,27
1,98,1,1.0,1.0,1.0,1.0,1,16,16,1
2,105,294,294.0,294.0,132.0,NaN,2,15,12,162
3,218,343,343.0,340.0,154.0,14.0,6,16,12,162
4,225,30,NaN,NaN,NaN,NaN,1,7,7,30


In [170]:
from itertools import groupby
from operator import itemgetter
# 连续多少天启动
def get_continuous(data):
    ranges = []
    for k, g in groupby(enumerate(data), lambda x: x[0] - x[1]):
        group = (map(itemgetter(1), g))
        group = list(map(int, group))
        ranges.append(group[-1] - group[0] + 1)
    return max(ranges)

In [175]:
%%time
continuous_launch_day = activity.groupby(['user_id'])['activity_day'].agg([get_continuous])
continuous_launch_day.head()

Wall time: 6.16 s


In [176]:
continuous_launch_day.head()

,get_continuous
user_id,
16,2
98,1
105,2
218,2
225,1


In [181]:
# 最后一天是否有拍摄video的行为
def whether_video(df, video, end):
    video = video[video.create_day == end]
    video = video.drop_duplicates()
    video.loc[video.create_day > 0, 'create_day'] = 1
    video.rename(columns={'create_day': 'whether_last_video'}, inplace=True)
    df = pd.merge(df, video, how='left', on='user_id')
    df['whether_last_video'] = df['whether_last_video'].fillna(0).astype(int)
    return df

In [182]:
df = pd.DataFrame()
df['user_id'] = np.unique(temp_create['user_id'])
test2 = whether_video(df, temp_create, end_day)

In [183]:
test2.head()

,user_id,whether_last_video
0,98,1
1,973,0
2,1014,1
3,1296,1
4,1938,0


In [199]:
def clear_variable():
    global launch_day_df
    global launch_day_gap_df
    launch_day_df = pd.DataFrame()
    launch_day_gap_df = pd.DataFrame()

In [194]:
import stats as sts
# 得到序列特征
def get_seq_feature(seq, seq_name, user_id):
    # total 11 features
    if not seq:
        print('seq is empty!')
        return
    df = pd.DataFrame()
    df[seq_name + '_mean'] = [np.mean(seq)]
    df[seq_name + '_median'] = [np.median(seq)]
    df[seq_name + '_max'] = [np.max(seq)]
    df[seq_name + '_min'] = [np.min(seq)]
    df[seq_name + '_var'] = [np.var(seq)]
    df[seq_name + '_std'] = [np.std(seq)]
    if len(seq) == 1:
        df[seq_name + '_upquantile'] = seq[0]
        df[seq_name + '_downquantile'] = 0
    else:
        df[seq_name + '_upquantile'] = [sts.quantile(seq, p=0.75)]
        df[seq_name + '_downquantile'] = [sts.quantile(seq, p=0.25)]
    if np.mean(seq) != 0: df[seq_name + '_discrete'] = [np.std(seq) / np.mean(seq)]
    else: df[seq_name + '_discrete'] = [np.NaN]
    try: df[seq_name + 'skew'] = [sts.skewness(seq)]
    except: df[seq_name + 'skew'] = [np.NaN]
    try: df[seq_name + 'kurt'] = [sts.kurtosis(seq)]
    except: df[seq_name + 'kurt'] = [np.NaN]
    df['user_id'] = [user_id]
    return df

In [195]:
# lambda 引用的函数
def get_feature(line, other_name, start):
    global launch_day_df
    global launch_day_gap_df
    user_id = line['user_id'].tolist()[0]
    seq = sorted(line['activity_day'].tolist())
    seq = [x - start for x in seq]
    # launch day 作为序列
    each_launch_day = get_seq_feature(seq, other_name, user_id)
    launch_day_df = pd.concat([launch_day_df, each_launch_day])
    # launch day gap 作为序列
    seq = np.unique(seq).tolist()
    seq_gap = []
    if len(seq) < 2:
        seq_gap = [0]
    else:
        for i in range(1, len(seq)):
            seq_gap.append(seq[i] - seq[i - 1])

    each_launch_gap_day = get_seq_feature(seq_gap, other_name + '_gap', user_id)
    launch_day_gap_df = pd.concat([launch_day_gap_df, each_launch_gap_day])

In [196]:
# action , page 0 1 2 3 4 , action_type 0 1 2 3 4 5 , user_id = author_id 发生的天数
def action_seq(df, action, start, end):
    print('提取action序列特征...')
    # 总 action 发生在哪一天的序列 和 间隔序列 特征
    action.groupby('user_id').apply(lambda x: get_feature(x, 'all_action', start))
    df = pd.merge(df, launch_day_df, how='left', on='user_id')
    df = pd.merge(df, launch_day_gap_df, how='left', on='user_id')
    clear_variable()
    df = df.drop_duplicates()
    return df#.to_csv('feature_set/%d_%d_action_seq.csv' % (start, end))

In [200]:
%%time
launch_day_df = pd.DataFrame()
launch_day_gap_df = pd.DataFrame()
df = pd.DataFrame()
df['user_id'] = np.unique(activity['user_id'])
test3 = action_seq(df,activity,start_day,end_day)

提取action序列特征...


D:\ProgramData\Anaconda3\lib\site-packages\stats.py:1614: RuntimeWarning: invalid value encountered in double_scalars
  n, total = _generalised_sum(data, lambda x: ((x-m)/s)**3)
D:\ProgramData\Anaconda3\lib\site-packages\stats.py:1660: RuntimeWarning: invalid value encountered in double_scalars
  n, total = _generalised_sum(data, lambda x: ((x-m)/s)**4)


Wall time: 4min 45s


In [ ]:
activity[activity['user_id']==16]

In [204]:
test3.columns

Index(['user_id', 'all_action_mean', 'all_action_median', 'all_action_max',
       'all_action_min', 'all_action_var', 'all_action_std',
       'all_action_upquantile', 'all_action_downquantile',
       'all_action_discrete', 'all_actionskew', 'all_actionkurt',
       'all_action_gap_mean', 'all_action_gap_median', 'all_action_gap_max',
       'all_action_gap_min', 'all_action_gap_var', 'all_action_gap_std',
       'all_action_gap_upquantile', 'all_action_gap_downquantile',
       'all_action_gap_discrete', 'all_action_gapskew', 'all_action_gapkurt'],
      dtype='object')

In [206]:
temp_launch[temp_launch['user_id']==16]

,user_id,launch_day
44140,16,13
44141,16,14
212269,16,15


In [208]:
def page_seq(df, action, start, end):
    for i in range(5):
        print('提取page_%d序列特征...' % i)
        action[action.page == i].groupby('user_id').apply(lambda x: get_feature(x, 'page_%d' % i, start))
        df = pd.merge(df, launch_day_df, how='left', on='user_id')
        df = pd.merge(df, launch_day_gap_df, how='left', on='user_id')
        clear_variable()
    df = df.drop_duplicates()
    return df#.to_csv('feature_set/%d_%d_page_seq.csv' % (start, end))

In [210]:
%%time
launch_day_df = pd.DataFrame()
launch_day_gap_df = pd.DataFrame()
df = pd.DataFrame()
df['user_id'] = np.unique(activity['user_id'])
test4 = page_seq(df, activity, start_day, end_day)

提取page_0序列特征...


D:\ProgramData\Anaconda3\lib\site-packages\stats.py:1614: RuntimeWarning: invalid value encountered in double_scalars
  n, total = _generalised_sum(data, lambda x: ((x-m)/s)**3)
D:\ProgramData\Anaconda3\lib\site-packages\stats.py:1660: RuntimeWarning: invalid value encountered in double_scalars
  n, total = _generalised_sum(data, lambda x: ((x-m)/s)**4)


提取page_1序列特征...
提取page_2序列特征...
提取page_3序列特征...
提取page_4序列特征...
Wall time: 11min 51s


In [211]:
test4.head()

,user_id,page_0_mean,page_0_median,page_0_max,page_0_min,page_0_var,page_0_std,page_0_upquantile,page_0_downquantile,page_0_discrete,...,page_4_gap_median,page_4_gap_max,page_4_gap_min,page_4_gap_var,page_4_gap_std,page_4_gap_upquantile,page_4_gap_downquantile,page_4_gap_discrete,page_4_gapskew,page_4_gapkurt
0,16,12.805556,13.0,14.0,12.0,0.601080,0.775294,13.0,12.0,0.060544,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
64,98,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
65,105,11.808824,11.0,13.0,11.0,0.963452,0.981556,13.0,11.0,0.083121,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
69,218,11.695652,11.0,14.0,11.0,1.603025,1.266106,11.0,11.0,0.108254,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN
73,225,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN


In [214]:
temp_launch.head()


,user_id,launch_day
44140,16,13
44141,16,14
212269,16,15
223219,98,16
125555,105,12


In [219]:
test5 = activity[(activity['user_id']==16) & (activity['page']==0)]
test5['activity_day']=test5['activity_day']-1
test5.head()

D:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,user_id,activity_day,page,video_id,author_id,action_type
7209577,16,12,0,3630860,1184452,0
7265612,16,12,0,1718686,1073012,0
7279247,16,12,0,575624,738746,0
7337159,16,12,0,2163185,280089,0
7337161,16,12,0,2163185,280089,0


In [221]:
np.median(test5['activity_day'])

13.0

In [232]:
def downquantile(column):
    return column.quantile(.25)


def upquantile(column):
    return column.quantile(.75)
# 偏度Skewness
def skew(column):
    return stats.skew(column)


# 峰度kurtosis
def kurt(column):
    return stats.kurtosis(column)


def discrete(column):
    if np.mean(column) != 0:
        return np.std(column) / np.mean(column)
    else:
        return np.NaN

In [240]:
%%time
func = ['mean', 'median', 'max', 'min', 'var', 'std', skew, kurt, discrete,upquantile,downquantile]
activity['activity_day'] = activity['activity_day'] - start_day
create_fun = activity.groupby('user_id')['activity_day'].agg(func).reset_index()
create_fun = create_fun.rename(
    columns={'mean': 'all_action_mean', 'median': 'all_action_median', 'max': 'all_action_max', 'min': 'all_action_min',
             'skew': 'all_action_skew', 'std': 'all_action_std', 'var': 'all_action_var', 'kurt': 'all_action_kurt',
             'discrete': 'all_action_discrete','upquantile':'all_action_upquantile','downquantile':'all_action_downquantile'})

D:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


Wall time: 19 s


In [241]:
create_fun.head(10)

,user_id,all_action_mean,all_action_median,all_action_max,all_action_min,all_action_var,all_action_std,all_action_skew,all_action_kurt,all_action_discrete,all_action_upquantile,all_action_downquantile
0,16,12.684211,13.0,14,12,0.541353,0.735767,0.566074,-0.946940,0.057495,13.0,12.0
1,98,15.000000,15.0,15,15,NaN,NaN,0.000000,-3.000000,0.000000,15.0,15.0
2,105,11.942177,11.0,14,11,1.133164,1.064502,0.336285,-1.640407,0.088986,13.0,11.0
3,218,12.425656,12.0,15,10,2.262732,1.504238,0.207646,-1.739610,0.120882,14.0,11.0
4,225,6.000000,6.0,6,6,0.000000,0.000000,0.000000,-3.000000,0.000000,6.0,6.0
5,336,5.727273,10.0,10,0,24.338558,4.933412,-0.280141,-1.915720,0.856481,10.0,0.0
6,607,0.000000,0.0,0,0,0.000000,0.000000,0.000000,-3.000000,NaN,0.0,0.0
7,711,5.000000,5.0,5,5,0.000000,0.000000,0.000000,-3.000000,0.000000,5.0,5.0
8,746,13.000000,13.0,13,13,0.000000,0.000000,0.000000,-3.000000,0.000000,13.0,13.0
9,901,13.000000,13.0,13,13,0.000000,0.000000,0.000000,-3.000000,0.000000,13.0,13.0


In [242]:
def get_sequence_feature(df, start, agg_feature, feature_name):
    func = ['mean', 'median', 'max', 'min', 'var', 'std', skew, kurt, discrete, upquantile, downquantile]
    sequence_feature_diff = pd.concat([df['user_id'], df.groupby(['user_id']).diff().rename({agg_feature: agg_feature + '_diff'}, axis=1)], axis=1)
    df[agg_feature] = df[agg_feature] - start
    sequence_feature = df.groupby('user_id')[agg_feature].agg(func).reset_index()
    sequence_feature = sequence_feature.rename(
        columns={'mean': feature_name + '_mean', 'median': feature_name + '_median', 'max': feature_name + '_max', 'min': feature_name + '_min',
                 'skew': feature_name + '_skew', 'std': feature_name + '_std', 'var': feature_name + '_var', 'kurt': feature_name + '_kurt',
                 'discrete': feature_name + '_discrete', 'upquantile': feature_name + '_upquantile', 'downquantile': feature_name + '_downquantile'})

    sequence_diff_feature = sequence_feature_diff.groupby('user_id')[agg_feature + '_diff'].agg(func).reset_index()
    sequence_diff_feature = sequence_diff_feature.rename(
        columns={'mean': feature_name + '_gap_mean', 'median': feature_name + '_gap_median', 'max': feature_name + '_gap_max', 'min': feature_name + '_gap_min',
                 'skew': feature_name + '_gap_skew', 'std': feature_name + '_gap_std', 'var': feature_name + '_gap_var', 'kurt': feature_name + '_gap_kurt',
                 'discrete': feature_name + '_gap_discrete', 'upquantile': feature_name + '_gap_upquantile', 'downquantile': feature_name + '_gap_downquantile'})
    sequence_feature = pd.merge(sequence_feature, sequence_diff_feature, how='left', on='user_id')
    return sequence_feature

In [249]:
%%time
action_seq_fea = get_sequence_feature(activity, start_day, 'activity_day', 'all_action')

D:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


Wall time: 40.8 s


In [250]:
action_seq_fea.head()

,user_id,all_action_mean,all_action_median,all_action_max,all_action_min,all_action_var,all_action_std,all_action_skew,all_action_kurt,all_action_discrete,...,all_action_gap_median,all_action_gap_max,all_action_gap_min,all_action_gap_var,all_action_gap_std,all_action_gap_skew,all_action_gap_kurt,all_action_gap_discrete,all_action_gap_upquantile,all_action_gap_downquantile
0,16,12.684211,13.0,14,12,0.541353,0.735767,0.566074,-0.946940,0.057495,...,0.0,1.0,0.0,0.035065,0.187256,NaN,NaN,5.196152,0.0,0.0
1,98,15.000000,15.0,15,15,NaN,NaN,0.000000,-3.000000,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,105,11.942177,11.0,14,11,1.133164,1.064502,0.336285,-1.640407,0.088986,...,0.0,2.0,0.0,0.017018,0.130453,NaN,NaN,12.719189,0.0,0.0
3,218,12.425656,12.0,15,10,2.262732,1.504238,0.207646,-1.739610,0.120882,...,0.0,1.0,0.0,0.014448,0.120201,NaN,NaN,8.209750,0.0,0.0
4,225,6.000000,6.0,6,6,0.000000,0.000000,0.000000,-3.000000,0.000000,...,0.0,0.0,0.0,0.000000,0.000000,NaN,NaN,NaN,0.0,0.0


In [202]:
test3.head(10)

,user_id,all_action_mean,all_action_median,all_action_max,all_action_min,all_action_var,all_action_std,all_action_upquantile,all_action_downquantile,all_action_discrete,...,all_action_gap_median,all_action_gap_max,all_action_gap_min,all_action_gap_var,all_action_gap_std,all_action_gap_upquantile,all_action_gap_downquantile,all_action_gap_discrete,all_action_gapskew,all_action_gapkurt
0,16,12.684211,13.0,14,12,0.531856,0.729285,13,12,0.057495,...,1.0,1,1,0.00,0.0,1,1,0.000000,NaN,NaN
4,98,15.000000,15.0,15,15,0.000000,0.000000,15,0,0.000000,...,0.0,0,0,0.00,0.0,0,0,NaN,NaN,NaN
5,105,11.942177,11.0,14,11,1.129310,1.062690,13,11,0.088986,...,1.5,2,1,0.25,0.5,2,1,0.333333,0.0,NaN
6,218,12.425656,12.0,15,10,2.256135,1.502044,14,11,0.120882,...,1.0,1,1,0.00,0.0,1,1,0.000000,NaN,NaN
7,225,6.000000,6.0,6,6,0.000000,0.000000,6,6,0.000000,...,0.0,0,0,0.00,0.0,0,0,NaN,NaN,NaN
8,336,5.727273,10.0,10,0,24.061983,4.905302,10,0,0.856481,...,5.0,9,1,16.00,4.0,9,1,0.800000,0.0,NaN
9,607,0.000000,0.0,0,0,0.000000,0.000000,0,0,NaN,...,0.0,0,0,0.00,0.0,0,0,NaN,NaN,NaN
10,711,5.000000,5.0,5,5,0.000000,0.000000,5,5,0.000000,...,0.0,0,0,0.00,0.0,0,0,NaN,NaN,NaN
11,746,13.000000,13.0,13,13,0.000000,0.000000,13,13,0.000000,...,0.0,0,0,0.00,0.0,0,0,NaN,NaN,NaN
12,901,13.000000,13.0,13,13,0.000000,0.000000,13,13,0.000000,...,0.0,0,0,0.00,0.0,0,0,NaN,NaN,NaN


In [5]:
temp_create.head()

,user_id,create_day
0,720497,1
1,720497,1
2,720497,1
3,1075211,6
4,1075211,12


In [7]:
temp_activity.head()

,user_id,activity_day,page,video_id,author_id,action_type
496070,16,13,3,2911602,245430,0
522440,16,13,3,1066348,245430,0
522441,16,13,3,1066348,245430,0
815812,16,13,3,1277921,280089,0
1289214,16,13,3,2565669,280089,0


In [ ]:
def get_my_fea(row,action):
    

In [ ]:
temp_create.groupby.apply(get_my_fea,)